
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan


# Mini station Météo 3 capteurs

Ce cahier permet d'expérimenter le pilotage d'une station munie de 3 capteurs :
 * BME280 température pression hygrométrie
 * TSL2561 luminosité
 * MCP9808 (température de précision)

Figure du montage :
![Montage ministation météo](images/bus_ministation_schema_bb.png)
![Licence Charles-Henri Eyraud](images/Licence.jpg)

Photo du Montage :
![Montage ministation météo](images/bus_ministation_photo.jpg)
![Licence Charles-Henri Eyraud](images/Licence.jpg)


**Bloc1** : déclaration des bibliothèques de programmes et des pilotes de capteurs

In [1]:
import mraa, upm
import _thread, time, math

from upm import pyupm_mcp9808 as mcp
from upm import pyupm_bmp280 as bme
from upm import pyupm_tsl2561 as tsl

**Bloc2** : déclaration des constantes statiques, adresses du bus et des capteurs utilisés; de l'altitude du lieu et des paramètres de contrôle du capteur de précision voir [site de description du pilote](https://iotdk.intel.com/docs/master/upm/classupm_1_1_m_c_p9808.html).

In [2]:
# Constant declarations
#----------------------
# i2c bus 1 on RaspBerry Pi
bus1 = 0
mcpAddr = 0x18
bmeAddr = 0x77
tslAddr = 0x39

localAlt = 169 #Altitude de Lyon
UPPER_TEMP = 0x02
LOWER_TEMP = 0x03
CRIT_TEMP = 0x04


**Bloc3** : Déclaration des 3 capteurs utilisés et choix de leur nom : temp, hygro, luxtls.

In [3]:
# create sensors on the bus 1
temp = mcp.MCP9808(bus1,mcpAddr)
hygro = bme.BME280(bus1,bmeAddr)
luxtsl = tsl.TSL2561(bus1,tslAddr)

**Bloc4** : Déclaraton des fonctions de pilotage ou de configuration des capteurs.

In [4]:
# Sensor Functions
#---------
# bme 280
#****************
def get_bmSensor(thisSensor) :
    name = thisSensor.getModuleName ()
    bmpId = thisSensor.getChipID ()
    print ('capteur : %s\t'%name + 'Id : %d\n'%bmpId)
    return

def reset_bmSensor(thisSensor) :
    thisSensor.reset()
    return

def mes_bmTPA(thisSensor) :
    thisSensor.update ()
    bmTemp = thisSensor.getTemperature ()
    bmPress = thisSensor.getPressure () /100.0
    bmAlt = thisSensor.getAltitude ()
    return(bmTemp, bmPress, bmAlt)

def mes_bmeHum(thisSensor) :
    thisSensor.update ()
    bmeHum = thisSensor.getHumidity ()
    return(bmeHum)

def convertSeaLevel (m_Pressure,m_Temperature,m_alt):
#    s_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    s_Pressure = m_Pressure * math.pow((1 - 0.0065 * m_alt / ( m_Temperature + 0.0065 * m_alt + 273.15)), -5.257) 
#    s_Pressure = m_Pressure / math.pow((1 - 0.0065 * m_alt / m_Temperature), 5.255 )
    return s_Pressure

# MCP9808
#********
def get_mcpSensor(thisSensor):
    unity = thisSensor.isCelsius()
    upper = thisSensor.getMonitorReg (UPPER_TEMP)
    lower = thisSensor.getMonitorReg (LOWER_TEMP)
    critic = thisSensor.getMonitorReg (CRIT_TEMP)
    hyst = thisSensor.getHysteresis ()
    resol = thisSensor.getResolution ()
    idManuf = thisSensor.getManufacturer ()
    mcpId = thisSensor.getDevicedId ()
    if unity :
        print ('We are measuring in Celsius')
    else :
        print ('We are measuring in Fahrenheit')
    print ('\tLower T° : %.3f\n'%lower + '\tUpperT° : %.3f\n'%upper + '\tCritical T° : %.3f\n'%critic)
    print ('Hysteresis value : %.3f'%hyst)
    print ('Resolution : %.4f'%resol)
    print ('Manufacturer Id : %d'%idManuf)
    print ('Device Id : %d\n'%mcpId)
    return

def mes_mcpTemp(thisSensor):
    mcpTemp = thisSensor.getTemp ()
    return (mcpTemp)

# TSL2561
#*******
def mes_tslLux(thisSensor) :
    lux = thisSensor.getLux ()
    return(lux)

def input_thread(CheckInput):
    check = input()
    CheckInput.append(check)
    return

def init_Thread () :
    CheckInput = []
    _thread.start_new_thread(input_thread, (CheckInput,))
    print ('Instructions :  \t"stop" pour arrêter les mesures.\n')
    print ('\t\t"resetBME" pour redémarrer le capteur BME280')
    return (CheckInput)
   
#Activate or test some sensors
#---------------------
#mcp.shutDown(False)


**Bloc5** : Fonction principale comprenant un bloc de dialogue pour recevoir des commandes en ligne

In [7]:
# Main function
###############
def measureMeteoParams():
    nbmes = 0
    while True :
        option1 = input('Instructions :  \t"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.\n--> ')
        if option1 == 'exit' :
            break
        elif option1 == 'start' :
            break
        elif option1 == 'check' :
            option2 = 'ready'
            while option2.strip != exit :
                option2 = input('Taper "info" pour avoir des informations, "changer" pour modifier les paramètres des capteurs, "exit" pour sortir.\n---> ')
                if option2 == 'exit':
                    break
                elif option2 == 'info':
                    option3 = 4
                    while option3 != 0 :
                        option3 = input('Informations sur les capteurs : BME280 (1) MCP9808 (2) exit (0)\n----> ')
                        if option3 == '0' :
                            break
                        elif option3 == '1' :
                            get_bmSensor(hygro)
                        elif option3 == '2' :
                            get_mcpSensor(temp)
                        else :
                            print ('Code inconnu : %s'%option3)
                elif option2 == 'changer' :
                    option3 = 3
                    while option3 != 0 :
                        option3 = input('Changement des paramètres des capteurs :\n\tMCP9808-precision (1)\n\tMCP9808-hysteresis (2)\n\tBME280_mode (3)\n\t sortir (0)\n----> ')
                        if option3 == '0' :
                            break
                        elif option3 == '1' :
                            val = input("precision value (LOW MED HI MAX) : ")
                            if val == 'LOW' :
                                temp.setResolution (0x00)
                            elif val == 'MED' :
                                temp.setResolution (0x01)
                            elif val == 'HI' :
                                temp.setResolution (0x02)
                            elif val == 'MAX' :
                                temp.setResolution (0x03)
                            else :
                                print ('Commande inconnue : "%s"\n'%val)
                        elif option3 == '2' :
                            val = input("Hysteresis value  (HYST_0 HYST_1_5 HYST_3_0 HYST_6_0) : ")
                            if val == 'HYST_0' :
                                temp.setHysteresis (0x0000)
                            elif val == 'HYST_1_5' :
                                temp.setHysteresis (0x0002)
                            elif val == 'HYST_3_0' :
                                temp.setHysteresis (0x0004)
                            elif val == 'HYST_6_0' :
                                temp.setHysteresis (0x0006)
                            else :
                                print ('Commande inconnue : "%s"\n'%val)
                        elif option3 == '3' :
                            val = input("precision value (0 1 2 3 4 5 ) : ")
                            if val == '0' :
                                hygro.setUsageMode(0)
                            elif val == '1' :
                                hygro.setUsageMode(1)
                            elif val == '2' :
                                hygro.setUsageMode(2)
                            elif val == '3' :
                                hygro.setUsageMode(3)
                            elif val == '4' :
                                hygro.setUsageMode(4)
                            elif val == '5' :
                                hygro.setUsageMode(5)
                            else :
                                print ('Code inconnu : %s'%val)
                        else :
                            print ('Code inconnu : "%s"\n'%option3)
                else :
                        print ('Commande inconnue : "%s"\n'%option2)
        else :
            print ('Commande inconnue : "%s"\n'%option1)
# Main loop
    if option1.strip() == 'start' :
        CheckInput = init_Thread()
        while True :
            if CheckInput == ['stop']: 
                print ('Arrêt demandé.')
                break
            elif CheckInput == ['resetBME'] :
                reset_bmSensor(hygro)
                print ('Reset BME effectué.')
                CheckInput.remove('resetBME')
                thread.start_new_thread(input_thread, (CheckInput,))
            elif CheckInput != [] :
                print ('Commande inconnue : "%s"\n'%CheckInput)
                thread.start_new_thread(input_thread, (CheckInput,))
                del CheckInput[:]
            else :
                time.sleep (0.5)
            tempHR = mes_mcpTemp(temp)
            hygroBME = mes_bmeHum(hygro) 
            [tempBME,pressBME,altBME] = mes_bmTPA(hygro)
            seaLevelPress = convertSeaLevel (pressBME,tempBME,localAlt)
            lux = mes_tslLux(luxtsl)
            if nbmes == 0 :
                print ('Int. Lum. : \tHumid(BME) : \tTemp(MCP) : \tTemp(BME) :  \tAlt(BME) : \tPress(BME) : \tPress(SealevelBME) : ' )
                nbmes += 1
            elif nbmes == 9 :
                nbmes = 0
            else :
                nbmes += 1
            print ('{0:3.2f} L\t{1:2.1f} %\t\t{2:.4f} °C\t{3:.2f} °C\t{4:.2f} m\t{5:.2f} hPa\t\t{6:.2f} hPa'.format(lux,hygroBME,tempHR,tempBME,altBME,pressBME,seaLevelPress))
            time.sleep (5)
    else :
        print ('Bye...')
    return

measureMeteoParams()


Instructions :  	"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.
--> start
Instructions :  	"stop" pour arrêter les mesures.

		"resetBME" pour redémarrer le capteur BME280
Int. Lum. : 	Humid(BME) : 	Temp(MCP) : 	Temp(BME) :  	Alt(BME) : 	Press(BME) : 	Press(SealevelBME) : 
234.00 L	49.0 %		20.1875 °C	21.68 °C	162.77 m	993.84 hPa		1013.46 hPa
235.00 L	49.1 %		20.1875 °C	21.68 °C	162.73 m	993.85 hPa		1013.47 hPa
15.00 L	51.7 %		20.3750 °C	21.71 °C	162.54 m	993.87 hPa		1013.49 hPa
12.00 L	52.8 %		20.5625 °C	21.78 °C	162.69 m	993.85 hPa		1013.46 hPa
9.00 L	55.2 %		20.7500 °C	21.86 °C	162.84 m	993.83 hPa		1013.44 hPa
234.00 L	52.4 %		20.6875 °C	21.87 °C	162.94 m	993.82 hPa		1013.43 hPa
99.00 L	50.1 %		20.5625 °C	21.87 °C	162.70 m	993.85 hPa		1013.46 hPa
231.00 L	49.5 %		20.6250 °C	21.86 °C	162.91 m	993.82 hPa		1013.43 hPa
232.00 L	49.3 %		20.5000 °C	21.85 °C	163.14 m	993.80 hPa		1013.41 hPa
221.00 L	49.2 %		20.5000 °C	21.85 °C	162.90 m	993.82 hPa		1013.43 